In [1]:
import numpy as np 
import pandas as pd 
import torch 
import scipy as sc
import stan
from src.df_processing import * 
import edward as ed

: 

In [122]:
# Define your function nu(mu, omega)
def nu_computation(mu, omega, tau):
    # Define your function here using mu and omega
    nu = np.linalg.inv(np.diag(np.exp(omega)))*(tau - mu)
    return nu  # Example function, replace with your actual function

def objective_fcts(): 
    # E_{N(0,I)}[∇_{theta}logp(x,θ)∇ T−1(ζ)+∇ log detJ −1(ζ)

    return fct_mu, fct_omega

def step_size(): 
    return 

def MC_integration(fct_obj, tirage, nu, nb_samples): 
    somme = 0
    for i in range(nb_samples): 
        nu_s = tirage(nu)
        somme += fct_obj(nu_s)
    return (1/nb_samples)*somme

In [3]:
def ppca(X, d):
    """
    Probabilistic Principal Component Analysis (PPCA).
    
    Parameters:
        X (numpy.ndarray): Input data matrix of shape (N, M), where N is the number of samples and M is the number of features.
        d (int): Number of principal components.
        
    Returns:
        tuple: Tuple containing the reconstructed data matrix, the projection matrix, and the noise variance.
    """
    N, M = X.shape

    # Center the data
    X_mean = np.mean(X, axis=0)
    X_centered = X - X_mean

    # Estimate covariance matrix
    cov_X = np.cov(X_centered, rowvar=False)

    # Perform eigenvalue decomposition
    eigenvalues, eigenvectors = np.linalg.eigh(cov_X)

    # Select the top d eigenvectors
    W = eigenvectors[:, -d:]

    # Estimate the noise variance
    sigma_squared = 1/(M - d) * np.sum(eigenvalues[:-d])

    # Compute the projection matrix
    M_inv = np.linalg.inv(np.dot(W.T, W) + sigma_squared * np.eye(d))
    C = np.dot(np.dot(W, M_inv), W.T)

    # Reconstruct the data
    X_reconstructed = np.dot(np.dot(C, X_centered.T), W.T) + X_mean.reshape(1, -1)

    return X_reconstructed.T, W, sigma_squared

In [119]:
# Model definition
def probas(N, M, D):
    # Priors
    z = sc.stats.multivariate_normal.rvs(mean=np.zeros(M), cov=np.eye(M, M), size=N)
    proba_z = sc.stats.multivariate_normal.pdf(z, mean=np.zeros(M), cov=np.eye(M, M)).prod()
    print("p_z: ",proba_z)

    alpha = sc.stats.invgamma.rvs(a=1,size=M)
    proba_alpha = sc.stats.invgamma.pdf(alpha, a=1).prod()
    print("p_alpha: ",proba_alpha)

    sigma = sc.stats.lognorm.rvs(loc=0, s=1, size=1)[0]
    proba_sigma = sc.stats.lognorm.pdf(sigma, loc=0, s=1)
    print("p_sigma: ",proba_sigma)

    w = sc.stats.multivariate_normal.rvs(mean=np.zeros(M), cov=sigma*np.diag(alpha), size=D)
    proba_w = sc.stats.multivariate_normal.pdf(w, mean=np.zeros(M), cov=sigma*np.diag(alpha)).prod()
    print("proba_w: ",proba_w)

    # likelihood
    lik = []
    proba_lik =1
    for i in range(N):
        obs = sc.stats.multivariate_normal.rvs(mean=np.dot(w,z[i]), cov=sigma*np.eye(D, D), size=1)         
        lik.append(obs)
        proba_lik *= sc.stats.multivariate_normal.pdf(obs, mean=np.dot(w,z[i]), cov=sigma*np.eye(D, D))

    print("proba_lik: ",proba_lik)
    priors = [z,alpha,sigma,w, lik]
    proba_priors = [proba_z, proba_alpha, proba_sigma, proba_w, proba_lik]

    #joint distribution 
    p_theta = proba_z*proba_alpha*proba_sigma*proba_w
    jd = p_theta*proba_lik
    print("p_jd: ",jd)

    return jd, priors, proba_priors


In [120]:
D = 5 # dimension 
M = 4 # maximum dimensions of latent space to consider 
N = 10 # number of data points in dataset 
jd, priors, proba_priors = probas(N, M, D)

p_z:  3.8858959628814784e-29
p_alpha:  0.0009597463589742657
p_sigma:  0.4874823900544616
proba_w:  1.561470073152198e-12
proba_lik:  2.975586420547798e-32
p_jd:  8.447200588225281e-76


In [6]:
def ADVI(x, ): 
    # Dataset x(1:N)
    N = len(x)
    # Model p(x,θ)
    i = 1 # Set iteration counter 

    # Parameter initialization 
    mu = torch.zeros(size= N, requires_grad=True)
    omega = torch.zeros(size= N, requires_grad=True) # Mean-field

    nu = torch.linalg.inv(torch.diag(torch.exp(w)))@(tau-mu)
    thr =  1e-4
    change_in_ELBO = 100
    # Define optimizer
    optimizer = torch.optim.SGD([mu, omega], lr=0.01)

    while True: 
        # Draw M samples from normal stamdard multivariate gaussian
        # Approximate gradients (wrt mu and w) with MC integration 
        # Compute nu
        nu = nu_computation(mu, omega, tau)

        L = objective_fct(nu, tau)

        # Backpropagation to compute gradients
        optimizer.zero_grad()
        L.backward()

        # Get gradients with respect to mu and omega
        gradient_mu = mu.grad
        gradient_omega = omega.grad
        
        # Update parameters
        optimizer.step()

        # Calculate step-size rho[i]
        rho = step_size(i, nu, rho)

        # Update mu and w 
        mu += np.diag(rho) @ gradient_mu
        omega += np.diag(rho) @ gradient_omega
        change_in_ELBO = np.abs()

        # increment iteration counter
        i +=1
        if abs(change_in_ELBO) < thr:
            break 

    return mu.item(), omega.item()

In [7]:
import torch
import torch.optim as optim

# Define your model using PyTorch
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        # Define your model architecture here
    
    def forward(self, x, theta):
        # Define the forward pass of your model
        return output

# Initialize your model
model = Model()

# Define your loss function
criterion = torch.nn.MSELoss()

# Define your optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Main optimization loop
threshold = 1e-6
while True:
    # Forward pass
    output = model(x, theta)
    
    # Compute the loss
    loss = criterion(output, target)
    
    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    
    # Update parameters
    optimizer.step()
    
    # Check convergence
    if abs(change_in_loss) < threshold:
        break

# Get the optimal values
optimal_values = model.parameters()


ValueError: optimizer got an empty parameter list

In [24]:
pystan_code = """
data { 
    int<lower=0> N; // number of data points in dataset
    int<lower=0> D; // dimension
    int<lower=0> M; // maximum dimension of latent space to consider
    vector[D] x[N]; // data
}

parameters {
    matrix[M, N] z; // latent variable
    matrix[D, M] w; // weights parameters
    real<lower=0> sigma; // standard deviation parameter
    vector<lower=0>[M] alpha; / hyper-parameters on weights
}

model {
// priors
    to_vector(z) ~ normal(0, 1); 
    for (d in 1:D)
        w[d] ~ normal(0, sigma * alpha);
    sigma ~ lognormal(0, 1);
    alpha ~ inv_gamma(1, 1);
    
    // likelihood
    for (n in 1:N)
        x[n] ~ normal(w * col(z, n), sigma);
}
"""

In [38]:
df = pd.read_csv('../taxi+service+trajectory+prediction+challenge+ecml+pkdd+2015/interpolation200.csv')

ls = [i.tolist() for i in extract_traj(df)]

code_data = {"N": len(ls),
                "D": 5,
                "M": 2,
                "x[N]": [[1,3,4,6,3] for i in range(len(ls))]}